In [1]:
import os
import zstandard
import json
import sys
import pandas as pd
import psutil

subreddit_list = ['AskReddit']#['politics', 'memes', 'teenagers', 'AmItheAsshole', 'AskReddit']
comment_fields_list = ['link_id', 'id', 'parent_id', 'author', 'author_fullname', 'permalink',
                       'body', 'total_awards_received', 'score', 'subreddit', 'removal_reason']
submission_fields_list = ['id', 'num_comments', 'title', 'author', 'author_fullname', 'permalink', 
                          'selftext', 'total_awards_received', 'score', 'subreddit', 'removal_reason']
data_path = 'data'

In [2]:
os.listdir(data_path)

['RC_2019-05.zst',
 'RC_2019-04.csv.gz',
 'RS_2019-06.zst',
 'RC_2019-02.zst',
 'RC_2018-11.zst',
 'RS_2019-01.zst',
 'RC_2019-10.csv.gz',
 'RC_2018-12.csv.gz',
 'RC_2019-03.zst',
 'RC_2019-09.csv.gz',
 'RS_2019-05.zst',
 'RC_2018-10.csv.gz',
 'RS_2019-10.zst',
 'RC_2019-05.csv.gz',
 'RC_2018-11.csv.gz',
 'RC_2019-11.csv.gz',
 'RC_2019-07.zst',
 'RC_2019-09.zst',
 'RS_2019-04.zst',
 'RC_2019-10.zst',
 'RS_2019-08.zst',
 'RC_2019-06.csv.gz',
 'RC_2019-06.zst',
 'RC_2019-01.zst',
 'RS_2019-12.zst',
 'RS_2018-11.zst',
 'RC_2019-08.zst',
 'RS_2019-07.zst',
 'RC_2018-12.zst',
 'RC_2019-08.csv.gz',
 'RC_2019-12.csv.gz',
 'RC_2019-01.csv.gz',
 'RC_2019-02.csv.gz',
 'RS_2019-03.zst',
 'RC_2019-11.zst',
 'RC_2018-10.zst',
 'RS_2019-09.zst',
 'RS_2019-02.zst',
 'RC_2019-12.zst',
 'RC_2019-04.zst',
 'RS_2019-11.zst',
 'RC_2019-07.csv.gz',
 'RC_2019-03.csv.gz',
 'RS_2018-12.zst']

In [3]:
rc_list = []
for f in os.listdir(data_path):
    if (f.find('RC')>=0) & (f.find('zst')>=0) & (f.replace('zst', 'csv.gz') not in os.listdir(data_path)):
        rc_list.append(os.path.join(data_path, f))
rc_list

[]

fl = os.path.join(data_path, 'RS_2018-11.zst')
tail = ''
j = 0
result = []
with open(fl, 'rb') as fh:
    dctx = zstandard.ZstdDecompressor()
    reader = dctx.stream_reader(fh)
    while True:
        j += 1
        chunk = reader.read(16384 * 2)
        if not chunk:
            break
        chunk = tail + chunk.decode()
        lst = chunk.split('\n')
        for i in range(len(lst) - 1):
            dct = json.loads(lst[i])
            if (dct['subreddit'] not in subreddit_list) | (dct['author'] == '[deleted]'):
                continue 
            print({i: dct.get(i, i) for i in submission_fields_list})
        if j > 100:
            break
        tail = lst[-1]

In [4]:
def preproc_fl(fl, subreddit_list, fields_list):
    tail = ''
    j = 0
    result = []
    with open(fl, 'rb') as fh:
        dctx = zstandard.ZstdDecompressor()
        reader = dctx.stream_reader(fh)
        while True:
            j += 1
            chunk = reader.read(16384 * 2)
            if not chunk:
                break
            chunk = tail + chunk.decode()
            lst = chunk.split('\n')
            for i in range(len(lst) - 1):
                dct = json.loads(lst[i])
                if (dct['subreddit'] not in subreddit_list) | (dct['author'] == '[deleted]'):
                    continue
                ans = [dct.get(i, None) for i in fields_list]
                result.append(ans)
            if j % 1000 == 0:
                print("\rEpisode {}, len of result {}, usage of memory {}.".format(
                    j, len(result), psutil.virtual_memory().percent), end="")
                sys.stdout.flush()
            tail = lst[-1]
            if psutil.virtual_memory().percent > 95:
                break
    print()
    return result

def preproc_fl_list(fl_list, subreddit_list, fields_list):
    res = []
    for fl in fl_list:
        res = preproc_fl(fl, subreddit_list, fields_list)
        df = pd.DataFrame(res, columns = fields_list)
        if 'body' in fields_list:
            df.body = df.body.apply(lambda x: str(x).replace(';', ','))
        if 'selftext' in fields_list:
            df.selftext = df.selftext.apply(lambda x: str(x).replace(';', ','))
        print(df.shape)
        df.to_csv(fl.replace('.zst', '.csv.gz'),
                  compression = 'gzip',
                  sep = ';',
                  index = False
                 )
    return 1

In [5]:
%%time
rc_res = preproc_fl_list(rc_list, subreddit_list, comment_fields_list)
print(rc_res)

1
CPU times: user 37 µs, sys: 4 µs, total: 41 µs
Wall time: 43.6 µs


%%time
df = pd.DataFrame(rc_res, columns = comment_fields_list)
df.body = df.body.apply(lambda x: str(x).replace(';', ','))
print(df.shape)

df.to_csv(os.path.join(data_path, 'rc_preprocessed.csv.gz'),
          compression = 'gzip',
          sep = ';',
          index = False
         )

In [6]:
rs_list = []
for f in os.listdir(data_path):
    #if (f.find('RS')>=0) & (f.find('zst')>=0):
    if (f.find('RS')>=0) & (f.find('zst')>=0) & (f.replace('zst', 'csv.gz') not in os.listdir(data_path)):
        rs_list.append(os.path.join(data_path, f))
rs_list

['data/RS_2019-06.zst',
 'data/RS_2019-01.zst',
 'data/RS_2019-05.zst',
 'data/RS_2019-10.zst',
 'data/RS_2019-04.zst',
 'data/RS_2019-08.zst',
 'data/RS_2019-12.zst',
 'data/RS_2018-11.zst',
 'data/RS_2019-07.zst',
 'data/RS_2019-03.zst',
 'data/RS_2019-09.zst',
 'data/RS_2019-02.zst',
 'data/RS_2019-11.zst',
 'data/RS_2018-12.zst']

In [7]:
%%time
rs_res = preproc_fl_list(rs_list, subreddit_list, submission_fields_list)
print(rs_res)

Episode 1841000, len of result 297883, usage of memory 8.6.
(297903, 11)
Episode 1545000, len of result 239925, usage of memory 8.9.
(240014, 11)
Episode 1815000, len of result 314310, usage of memory 9.0.
(314467, 11)
Episode 2067000, len of result 297029, usage of memory 9.1.
(297051, 11)
Episode 1784000, len of result 302266, usage of memory 9.1.
(302387, 11)
Episode 2090000, len of result 367115, usage of memory 9.2.
(367143, 11)
Episode 2084000, len of result 317647, usage of memory 9.3.
(317692, 11)
Episode 1266000, len of result 180164, usage of memory 9.1.
(180192, 11)
Episode 1971000, len of result 337181, usage of memory 9.0.
(337241, 11)
Episode 1783000, len of result 276047, usage of memory 9.1.
(276070, 11)
Episode 1944000, len of result 282044, usage of memory 9.1.
(282177, 11)
Episode 1480000, len of result 235029, usage of memory 9.1.
(235231, 11)
Episode 2039000, len of result 286074, usage of memory 9.1.
(286145, 11)
Episode 1388000, len of result 201500, usage of mem

%%time
df = pd.DataFrame(rs_res, columns = submission_fields_list)
df.selftext = df.selftext.apply(lambda x: str(x).replace(';', ','))
print(df.shape)

df.to_csv(os.path.join(data_path, 'rs_preprocessed.csv.gz'),
          compression = 'gzip',
          sep = ';',
          index = False
         )